# RNN Model

### Imports

In [23]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM

### Load data

In [2]:
covid = pd.read_csv("../datasets/WHO-COVID-19-global-data.csv")

In [32]:
covid.head(15)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0
5,2020-01-08,AF,Afghanistan,EMRO,0,0,0,0
6,2020-01-09,AF,Afghanistan,EMRO,0,0,0,0
7,2020-01-10,AF,Afghanistan,EMRO,0,0,0,0
8,2020-01-11,AF,Afghanistan,EMRO,0,0,0,0
9,2020-01-12,AF,Afghanistan,EMRO,0,0,0,0


### Create mini datasets of each country with the first day having >= 100 cases as the start date

In [3]:
covid_shifted = covid[covid['Cumulative_cases'] >= 100]

In [5]:
ls = []
for country in covid_shifted['Country'].unique():
# a dataframe of an individual country
# starting at the first day with over 100 cases
# with just the date and cumulative cases
# with the index reset
    temp_df = covid_shifted[(covid_shifted['Cumulative_cases'] >= 100) 
                            & (covid_shifted['Country'] == country)][['Date_reported', 'Cumulative_cases']].reset_index().drop(columns='index')
    temp_df['Days_since_100'] = [i for i in range(len(temp_df))]
# add it to the list as a tuple to keep country name while reducing df size
    ls.append((country, temp_df))

In [6]:
# view the first three rows of the first four country reports
for tup in ls[0:2]:
    print(tup[0])
    display(tup[1].head(5))

Afghanistan


,Date_reported,Cumulative_cases,Days_since_100
0,2020-03-28,106,0
1,2020-03-29,114,1
2,2020-03-30,114,2
3,2020-03-31,166,3
4,2020-04-01,192,4


Albania


,Date_reported,Cumulative_cases,Days_since_100
0,2020-03-25,108,0
1,2020-03-26,136,1
2,2020-03-27,148,2
3,2020-03-28,159,3
4,2020-03-29,174,4


### The model

In [24]:
# test country
country = ls[0]

step = 4

X = [val for val in country[1]['Cumulative_cases']]
# add "step" extra to be used to make the final pred
X = np.append(X, X[-1::]*step)


In [30]:
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1,step), activation="relu"))
# model.add(Dense(8, activation="relu")) 
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='rmsprop')
# model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (simple_rnn_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported